# WEB SCRAPPING IMDB TOP 250 MOVIES AND THERE CAST

### Required Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [ ]:
headers = {
    'User-Agent': 'Brave (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
}

In [ ]:
url=" https://www.imdb.com/chart/top/?ref_=nv_mv_250"

In [ ]:
response=requests.get(url,headers=headers) #requesting the url for accesssing data

In [ ]:
response #checking the connection status

In [ ]:
response.content

In [ ]:
soup=bs(response.content,'html.parser') #parsing the elements in html 

In [ ]:
soup

In [ ]:
soup.find_all('h3',class_='ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 eBRbsI compact-list-view ipc-metadata-list--base')

In [ ]:
movie_name=[]
for i in soup.find_all('h3'):
    movie_name.append(i.text)

In [ ]:
print(movie_name)  #as the web page is having js the beauitiful soup cannot handle the automation so prints 25 out of 250 as the content changes dynamically

# Using Selenium for Interaction With Dynamic Content 

In [ ]:
from selenium import webdriver as wd 


In [ ]:
driver=wd.Chrome()

In [ ]:
driver.get("https://www.imdb.com/chart/top/")

In [ ]:
soup=bs(driver.page_source,'html.parser')

In [ ]:
soup

In [ ]:
movie_data=[]
movie_datas=soup.find_all('li',class_='ipc-metadata-list-summary-item sc-10233bc-0 iherUv cli-parent')
for i in movie_datas:
    movie_data.append(i.text)
movie_data

In [ ]:
Title=[]
YT=[]
corrected_YT=[]
Rating=[]
Duration_with_year=[]
Duration=[]
for data in movie_data:
    title_match=re.search(r'\. (.+?)(?=\d{4})', data)
    title=title_match.group(1) if title_match else None
    year_match=re.search(r'\d{4}', data)
    year=year_match.group() if year_match else None
    duration_match = re.search(r'\d+h \d+m', data)
    duration = duration_match.group() if duration_match else None
    score_match = re.search(r'\d\.\d', data)
    score = score_match.group() if score_match else None
    Title.append(title)
    YT.append(str((year.split(" "))))
    Rating.append(score)
    Duration_with_year.append(duration)
for i in Duration_with_year:
    if i==None:
        Duration.append(None)
    else:
        Duration.append(i[4:])
for j in YT:
    corrected_YT.append(j[2:6])

In [ ]:
movie_titles=Title
release_years=YT
durations=Duration
ratings=Rating
assert len(movie_titles)==len(release_years)==len(durations)==len(ratings)
data={
'Title': movie_titles,
'Duration': durations,
'Score': ratings,
'Year':corrected_YT
}
df=pd.DataFrame(data)
csv_file = 'movies.csv'
df.to_csv(csv_file, index=False, encoding='utf-8')
print(f'Data successfully written to {csv_file}')


In [ ]:
df=pd.read_csv('movies.csv')

In [ ]:
df

In [ ]:
driver.quit()